In [10]:
import pandas as pd

# Read the CSV file into a DataFrame
usd_exchange_rate = pd.read_csv("EXCHANGE_RATES_2010-PRESENT.csv")

# Filter rows where the "Currency" column is equal to "USD"
usd_rows = usd_exchange_rate[usd_exchange_rate['Currency'] == 'USD']

# Display the filtered rows
print(usd_rows)

      Currency        Date  Buy Rate  Sell Rate
18393      USD  2023-10-12  318.0688   328.8289
18394      USD  2023-10-11  318.0267   328.7869
18395      USD  2023-10-10  317.9102   328.7491
18396      USD  2023-10-09  318.3050   328.9350
18397      USD  2023-10-06  318.2819   328.9108
...        ...         ...       ...        ...
21016      USD  2010-08-13  111.6162   113.2044
21017      USD  2010-07-19  113.5755   111.9651
21018      USD  2010-06-15  112.9046   114.4329
21019      USD  2010-05-11  112.8658   114.4100
21020      USD  2010-05-07  112.9725   114.5881

[2628 rows x 4 columns]


In [11]:
import pandas as pd
import os

# Initialize an empty list to store the dataframes
dataframes = []

# Specify the directory containing the CSV files
directory_path = "../../CSE_DATA/SHARE_CHANGE/"

filenames = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.endswith('.csv')]

# Initialize an empty DataFrame to store the data
data = pd.DataFrame()

# Loop through the CSV files and read them into DataFrames
for file in filenames:
    df = pd.read_csv(file)
    # Extract the label from the file name and remove the ".csv" extension
    label = os.path.basename(file).replace(".csv", "")
    # Add the label as a new column
    df['Label'] = label
    dataframes.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Filter out data associated with the label 'SWAD.N0000'
combined_df = combined_df[combined_df['Label'] != 'SWAD.N0000']

# Convert 'Trade Date' from MM/DD/YY to DD/MM/YYYY format
combined_df['Trade Date'] = pd.to_datetime(combined_df['Trade Date'], format='%m/%d/%y').dt.strftime('%d/%m/%Y')

# Split 'Trade Date' into 'Day', 'Month', and 'Year' columns
combined_df[['Day', 'Month', 'Year']] = combined_df['Trade Date'].str.split('/', expand=True)


# Select the columns 'Label', 'Day', 'Month', 'Year', and 'Close (Rs.)'
selected_columns = ['Label', 'Day', 'Month', 'Year', 'Close (Rs.)']
result_df = combined_df[selected_columns]

# Sort the DataFrame by 'Label' and the new date columns
result_df = result_df.sort_values(by=['Year', 'Month', 'Day', 'Label'])

print(result_df)

             Label Day Month  Year  Close (Rs.)
216103   JKH.N0000  17    08  1986          0.0
132130   SUN.N0000  30    03  1990         12.5
74593   GREG.N0000  02    04  1990          0.5
401097  SELI.N0000  10    01  1991        550.0
463724  SHAL.N0000  08    02  1991        575.0
...            ...  ..   ...   ...          ...
166672  VPEL.N0000  13    10  2023          6.8
336467  WAPO.N0000  13    10  2023         31.3
446639  WATA.N0000  13    10  2023         74.0
69288   WIND.N0000  13    10  2023         19.0
132589  YORK.N0000  13    10  2023        168.5

[630051 rows x 5 columns]


In [19]:
# Convert 'Year', 'Month', and 'Day' columns to strings in both DataFrames
result_df['Year'] = result_df['Year'].astype(str)
result_df['Month'] = result_df['Month'].astype(str)
result_df['Day'] = result_df['Day'].astype(str)

usd_exchange_rate['Year'] = usd_exchange_rate['Date'].dt.year.astype(str)
usd_exchange_rate['Month'] = usd_exchange_rate['Date'].dt.month.astype(str)
usd_exchange_rate['Day'] = usd_exchange_rate['Date'].dt.day.astype(str)

# Merge share_df with exchange_rate on Year, Month, and Day columns
merged_df = result_df.merge(usd_exchange_rate[['Day', 'Month', 'Year', 'Buy Rate', 'Sell Rate']], on=['Day', 'Month', 'Year'], how='left')

# Remove rows where both 'Buy Rate' and 'Sell Rate' are NaN
merged_df = merged_df.dropna(subset=['Buy Rate', 'Sell Rate'])

# Print or use merged_df
print(merged_df)


              Label Day Month  Year  Close (Rs.)  Buy Rate  Sell Rate
244282   AAIC.N0000  13    10  2010         70.0  109.1219   112.0223
244283   AAIC.N0000  13    10  2010         70.0  109.5084   112.3067
244284   AAIC.N0000  13    10  2010         70.0  115.6973   118.5271
244285   AAIC.N0000  13    10  2010         70.0  154.5356   158.1334
244286   AAIC.N0000  13    10  2010         70.0  175.3399   178.9376
...             ...  ..   ...   ...          ...       ...        ...
1029390  WIND.N0000  12    10  2023         18.6  336.5029   351.5075
1029391  WIND.N0000  12    10  2023         18.6  391.0078   406.8217
1029392  WIND.N0000  12    10  2023         18.6    2.1247     2.2172
1029393  WIND.N0000  12    10  2023         18.6  232.2532   242.9632
1029394  WIND.N0000  12    10  2023         18.6  318.0688   328.8289

[456680 rows x 7 columns]


In [20]:
# Save the filtered DataFrame to a CSV file
merged_df.to_csv('usd_vs_share_price_data.csv', index=False)